# set api

In [ ]:
import yaml
with open('/home/.dev.yaml','r') as f:
  config = yaml.safe_load(f)
print(config)

import os
os.environ["OPENAI_API_KEY"] = config['openai_api_key']
os.environ["SERPAPI_API_KEY"] = config['serpapi_api_key']


# prompt

In [ ]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(input_variables=["name"],template="{name}人口有多少？")
final_prompt=prompt.format(name="香港")

# llm

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import OpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
#chat = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0)
chat = ChatOpenAI(temperature=0)
openai = OpenAI()
#openai(prompt=prompt.format(name="美国"))
messages = [
    SystemMessage(content="结尾增加@hello@"),
    HumanMessage(content=prompt.format(name="美国"))
]
result1 = chat(messages)
print(result1)
result2 = chat.generate([messages])
print(result2)
result2.llm_output['token_usage']['total_tokens']

# chain

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=openai , prompt=prompt)
chain.run("香港")

# index

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("Python_lab5.pdf")
pages = loader.load_and_split()
pages[0]
from langchain.chains import RetrievalQA

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
urls = [
    "https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-8-2023",
    "https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-9-2023"
]
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

# agent

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
tools = load_tools(["serpapi","llm-math"],llm=openai)
agent = initialize_agent(tools, openai, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("世界人口多少")

# memory

In [ ]:
from langchain import ConversationChain
conversation = ConversationChain(llm=openai, verbose=True)
output = conversation.predict(input="Hi there!")
print(output)
output = conversation.predict(input="what do I say just now?")
print(output)


In [ ]:
from langchain.schema import AIMessage,HumanMessage,SystemMessage
messages=[SystemMessage(content="回答结尾说'🙏'"),HumanMessage(content="世界人口有多少")]
chat(messages)